# Generate Passport Index datasets
* Data by Passport Index 2023: https://www.passportindex.org/
* In both tidy and matrix formats
* Using ISO-2, ISO-3, and full country names

In [1]:
import requests
import json
import pandas as pd

In [2]:
codes = pd.read_csv(
    'https://gist.githubusercontent.com/ilyankou/b2580c632bdea4af2309dcaa69860013/raw/420fb417bcd17d833156efdf64ce8a1c3ceb2691/country-codes',
    dtype=str
).fillna('NA').set_index('ISO2')

def fix_iso2(x):
    o = {
        'UK': 'GB',
        'RK': 'XK'
    }
    return o[x] if x in o else x

## Get data from PassportIndex

In [3]:
# URL of the compare passport page
url = 'https://www.passportindex.org/comparebyPassport.php?p1=ro&p2=gt&p3=qa'

# Make a request to the .php page taht outputs data
result_raw = requests.post('https://www.passportindex.org/incl/compare2.php', headers={
    "Host": "www.passportindex.org",
                    "User-Agent": "Mozilla/5.0",
                    "Accept": "*/*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip, deflate",
                    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                    "X-Requested-With": "XMLHttpRequest",
                    "Content-Length": "9",
                    "Origin": "https://www.passportindex.org",
                    "DNT": "1",
                    "Connection": "keep-alive",
                    "Pragma": "no-cache",
                    "Cache-Control": "no-cache",
                    "TE": "Trailers",
}, data={
    'compare': '1'
})

## Clean up the data

In [4]:
result = json.loads( result_raw.text )
obj = {}

for passport in result:
    
    # Fix ISO-2 codes
    passport = fix_iso2(passport)
    
    # Add passport to the object
    if passport not in obj:
        obj[passport] = {}
    
    # Add destinations for the given passport
    for dest in result[passport]['destination']:
        
        text = dest['text']
        res = ''
        duration = dest['dur']
        
        # ** Visa required, incl Cuba's tourist card **
        if text == 'visa required' or text == 'tourist card':
            res = 'visa required'
        
        # ** Visa on arrival **
        elif 'visa on arrival' in text:
            res = 'visa on arrival'
            
        # ** Covid-19 ban ** 
        elif text == 'COVID-19 ban':
            res = 'covid ban'
            
        # ** Visa-free, incl. Seychelles' tourist registration **
        elif 'visa-free' in text or 'tourist registration' in text or 'visa waiver' in text:
            res = dest['dur'] if dest['dur'] != '' else 'visa free'
            
        # ** eVisas, incl eVisitors (Australia), eTourist cards (Suriname),
        # eTA (US), and pre-enrollment (Ivory Coast), or EVW (UK) **
        elif 'eVis' in text or 'eTourist' in text or text == 'eTA' or text == 'pre-enrollment' or text == 'EVW':
            res = 'e-visa'
            
        # ** No admission, including Trump ban **
        elif text == 'trump ban' or text == 'not admitted':
            res = 'no admission'
        
        # pad the duration
        if res != '' and duration != '':
            if 'visa-free' in text or 'tourist registration' in text or 'visa waiver' in text:
                continue
            res = res + ' (' + duration + ')'
            
        # Update the result!
        obj[passport][ fix_iso2(dest['code']) ] = res if res != '' else dest['text']
        

## Save

In [5]:
# ISO-2: Matrix
matrix = pd.DataFrame(obj).T.fillna(-1)
matrix.to_csv('passport-index-matrix-iso2.csv', index_label='Passport')

# ISO-2: Tidy
matrix.stack().to_csv(
    'passport-index-tidy-iso2.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])



# ISO-3: Matrix
iso2to3 =  { x:y['ISO3'] for x,y in codes.iterrows() }
matrix.rename(columns=iso2to3, index=iso2to3).to_csv('passport-index-matrix-iso3.csv', index_label='Passport')

# ISO-3: Tidy
matrix.rename(columns=iso2to3, index=iso2to3).stack().to_csv(
    'passport-index-tidy-iso3.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])


# Country names: Matrix
iso2name =  { x:y['Country'] for x,y in codes.iterrows() }
matrix.rename(columns=iso2name, index=iso2name).to_csv('passport-index-matrix.csv', index_label='Passport')

# Country names: Tidy
matrix.rename(columns=iso2name, index=iso2name).stack().to_csv(
    'passport-index-tidy.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])

In [6]:
# Print all values
tidy = matrix.rename(columns=iso2to3, index=iso2to3).stack()
tidy.value_counts()

visa required            13905
-1                       10872
e-visa                    4070
visa on arrival (30)      2750
visa free                 1869
e-visa (90)               1586
visa on arrival (90)      1377
e-visa (30)                780
visa on arrival            648
visa on arrival (45)       223
visa on arrival (150)      185
visa on arrival (15)       184
visa required (30)         158
visa required (90)         153
e-visa (14)                149
visa on arrival (60)       102
e-visa (60)                 75
e-visa (180)                62
e-visa (15)                 52
e-visa (120)                50
visa on arrival (42)        48
visa on arrival (31)        38
no admission                33
visa on arrival (120)       21
visa on arrival (14)         8
visa on arrival (7)          3
visa on arrival (21)         1
Name: count, dtype: int64

In [7]:
tidy[ tidy == 'no admission' ]

ARM  AZE    no admission
BGD  IRQ    no admission
     LBY    no admission
HTI  SUR    no admission
IRN  LBY    no admission
ISR  DZA    no admission
     BGD    no admission
     BRN    no admission
     IRN    no admission
     LBN    no admission
     LBY    no admission
     MYS    no admission
     PAK    no admission
     SAU    no admission
     SYR    no admission
     YEM    no admission
XKX  ARM    no admission
     CUB    no admission
     HKG    no admission
     SYC    no admission
PRK  JPN    no admission
     LKA    no admission
PAK  LBY    no admission
PSE  MDG    no admission
     SYR    no admission
PHL  KWT    no admission
SOM  AUS    no admission
     CAN    no admission
SDN  LBY    no admission
SYR  LBY    no admission
TWN  GEO    no admission
TJK  KGZ    no admission
YEM  LBY    no admission
dtype: object

### Difference with previous run

In [8]:
tidy_old = pd.read_csv('legacy/2023-12-03/passport-index-tidy-iso3.csv')

In [9]:
(tidy
 .to_frame()
 .reset_index()
 .merge(
     tidy_old, how='inner',
     left_on=['level_0', 'level_1'],
     right_on=['Passport', 'Destination']
 )
 .assign(
     is_different=lambda df_: df_[0].ne(df_.Requirement)
 )
 .query('is_different & (Passport != Destination)')
)

,level_0,level_1,0,Passport,Destination,Requirement,is_different
12,AFG,BGD,visa on arrival (30),AFG,BGD,visa on arrival,True
19,AFG,BOL,visa on arrival (90),AFG,BOL,visa on arrival,True
26,AFG,BDI,visa on arrival (30),AFG,BDI,visa on arrival,True
27,AFG,KHM,visa on arrival (30),AFG,KHM,visa on arrival,True
36,AFG,COM,visa on arrival (45),AFG,COM,visa on arrival,True
...,...,...,...,...,...,...,...
39394,ZWE,VNM,e-visa (90),ZWE,VNM,e-visa,True
39396,ZWE,ZMB,-1,ZWE,ZMB,90,True
39399,ZWE,DMA,-1,ZWE,DMA,21,True
39400,ZWE,HTI,-1,ZWE,HTI,90,True


In [10]:

import pandas as pd
import re

# Specify the paths for your three CSV files
input_file1 = 'passport-index-tidy.csv'  # Replace with your actual file path
input_file2 = 'passport-index-tidy-iso2.csv'  # Replace with your actual file path
input_file3 = 'passport-index-tidy-iso3.csv'  # Replace with your actual file path

output_file1 = 'passport-index-tidy.csv'  # Replace with your actual file path
output_file2 = 'passport-index-tidy-iso2.csv'  # Replace with your actual file path
output_file3 = 'passport-index-tidy-iso3.csv'  # Replace with your actual file path


# Read the CSV file into a DataFrame
# loop through the files    
for input_file, output_file in zip([input_file1, input_file2, input_file3], [output_file1, output_file2, output_file3]):
    df = pd.read_csv(input_file)

    # Define a function to extract numbers from brackets
    def extract_duration(text):       
        match = re.search(r'\((\d+)\)', str(text))
        if match:
            return int(match.group(1))
        else:
            return 0
        
        
    # remove the brackets from the text


    # Apply the function to create a new 'duration' column for each row
    df['duration'] = df.apply(lambda row: extract_duration(row), axis=1)

    df = df.assign(
        Requirement=df.Requirement.str.replace(r'\s*\(\d+\)', '', regex=True)
    )
    # Save the modified DataFrame to a new CSV file

    # give each row a unique identifier, starting from 1
    df['id'] = range(1, len(df) + 1)

    df.to_csv(output_file, index=False)

    